Date Last Modified: June 22th 2024 <br>
Date First Created: June 22th 2024

In [1]:
import numpy as np
import pandas as pd
import pickle
from tqdm import tqdm
from functools import reduce

# Import Connectors and Module Overlap data

In [2]:
directory = "/home/xavier/Desktop/Cell_interactome_stoichiometry/Yeast_Epistasis_Stoichiometries/Third_Run/"

with open((directory + "Python_Connector_Overlap/results/PPI_pairwise_module_shortest_paths_dict.pickle"), "rb") as handle:
    PPI_shortest_path_dict = pickle.load(handle)

In [3]:
with open((directory + "Python_Connector_Overlap/results/GI_pairwise_module_shortest_paths_dict.pickle"), "rb") as handle:
    GI_shortest_path_dict = pickle.load(handle)

In [21]:
# Import GI-PPI module overlap table
GI_PPI_Module_Overlap_DF = pd.read_csv(directory+"Python_Module_Overlap/results/GI_PPI_optimal_module_overlap_clean.csv", index_col = 0)

## Create a dictionary that maps GI module IDs to PPI module IDs
GI_Module_ls = [int(ID) for ID in GI_PPI_Module_Overlap_DF['Cluster_ID'].tolist() if ID != 'None']

conversion_dict = dict.fromkeys(GI_Module_ls)

for key in list(conversion_dict.keys()):
    index = GI_PPI_Module_Overlap_DF['Cluster_ID'].tolist().index(key)
    conversion_dict[key] = GI_PPI_Module_Overlap_DF['CPX_ID'].tolist()[index]

In [5]:
directory = "/home/xavier/Desktop/Cell_interactome_stoichiometry/Yeast_Epistasis_Stoichiometries/Third_Run/"

with open((directory + "Python_Connector_Overlap/results/decode_LUT_dict.pickle"), "rb") as handle:
    LUT = pickle.load(handle)

LUT

{0: 'Q0045',
 1: 'Q0085',
 2: 'Q0140',
 3: 'Q0160',
 4: 'Q0250',
 5: 'R0020C',
 6: 'R0040C',
 7: 'YAL001C',
 8: 'YAL002W',
 9: 'YAL003W',
 10: 'YAL004W',
 11: 'YAL005C',
 12: 'YAL007C',
 13: 'YAL008W',
 14: 'YAL009W',
 15: 'YAL010C',
 16: 'YAL011W',
 17: 'YAL012W',
 18: 'YAL013W',
 19: 'YAL014C',
 20: 'YAL015C',
 21: 'YAL016C-B',
 22: 'YAL016W',
 23: 'YAL017W',
 24: 'YAL018C',
 25: 'YAL019W',
 26: 'YAL020C',
 27: 'YAL021C',
 28: 'YAL022C',
 29: 'YAL023C',
 30: 'YAL024C',
 31: 'YAL025C',
 32: 'YAL026C',
 33: 'YAL027W',
 34: 'YAL028W',
 35: 'YAL029C',
 36: 'YAL030W',
 37: 'YAL031C',
 38: 'YAL032C',
 39: 'YAL033W',
 40: 'YAL034C',
 41: 'YAL034W-A',
 42: 'YAL035W',
 43: 'YAL036C',
 44: 'YAL037C-A',
 45: 'YAL037W',
 46: 'YAL038W',
 47: 'YAL039C',
 48: 'YAL040C',
 49: 'YAL041W',
 50: 'YAL042C-A',
 51: 'YAL042W',
 52: 'YAL043C',
 53: 'YAL044C',
 54: 'YAL044W-A',
 55: 'YAL045C',
 56: 'YAL046C',
 57: 'YAL047C',
 58: 'YAL048C',
 59: 'YAL049C',
 60: 'YAL051W',
 61: 'YAL053W',
 62: 'YAL054C',
 63:

# Check for overlap in GI and PPI connectors

In [6]:
shortest_path_overlap = {key1: {key2: [] for key2 in GI_shortest_path_dict[key1].keys()} for key1 in GI_shortest_path_dict.keys()}
shortest_path_overlap

{1: {2: [],
  3: [],
  4: [],
  5: [],
  6: [],
  7: [],
  8: [],
  9: [],
  10: [],
  11: [],
  12: [],
  13: [],
  14: [],
  15: [],
  16: [],
  17: [],
  18: [],
  19: [],
  20: [],
  21: [],
  22: [],
  23: [],
  24: [],
  25: [],
  26: [],
  27: [],
  28: [],
  29: [],
  30: [],
  31: [],
  32: [],
  33: [],
  34: [],
  35: [],
  36: [],
  37: [],
  38: [],
  39: [],
  40: [],
  41: [],
  42: [],
  43: [],
  44: [],
  45: [],
  47: [],
  48: [],
  49: [],
  50: [],
  51: [],
  52: [],
  53: [],
  54: [],
  55: [],
  56: [],
  57: [],
  58: [],
  59: [],
  60: [],
  61: [],
  62: [],
  63: [],
  64: [],
  65: [],
  66: [],
  67: [],
  68: [],
  69: [],
  70: [],
  71: [],
  72: [],
  73: [],
  74: [],
  75: [],
  76: [],
  77: [],
  78: [],
  79: [],
  80: [],
  81: [],
  82: [],
  83: [],
  84: [],
  85: [],
  86: [],
  87: [],
  88: [],
  89: [],
  90: [],
  91: [],
  92: [],
  93: [],
  94: [],
  95: [],
  96: [],
  97: [],
  98: [],
  99: [],
  100: [],
  101: [],
  102: [],
  

In [7]:
for key1 in tqdm(GI_shortest_path_dict.keys()):
    for key2 in GI_shortest_path_dict[key1].keys():
        
        if ((key1 not in list(conversion_dict.keys())) | (key2 not in list(conversion_dict.keys()))):
            continue
        
        if (conversion_dict[key1] == 'None') | (conversion_dict[key2] == 'None'):
            continue
        
        GI_paths = GI_shortest_path_dict[key1][key2]['Trimmed_Shortest_Path']
        PPI_paths = PPI_shortest_path_dict[conversion_dict[key1]][conversion_dict[key2]]['Trimmed_Shortest_Path']
        
        for GI_path in GI_paths:
            for PPI_path in PPI_paths:
                GI_PPI_Overlap = list(set(GI_path).intersection(PPI_path))
                GI_PPI_Overlap = np.array([x for x in GI_PPI_Overlap.copy() if x != -1], dtype = np.int16)
                if len(GI_PPI_Overlap) >= 1:
                    shortest_path_overlap[key1][key2].append(GI_PPI_Overlap)
                
                if len(GI_PPI_Overlap) > 1:
                    print(GI_PPI_Overlap)

100%|███████████████████████████████████████████████████████████████| 111/111 [35:04<00:00, 18.96s/it]


In [8]:
## Clean shortest path dictionary.
# 1. If there are no shortest paths between two modules, remove dict entry
# 2. If there are identical instances of shortest paths between two modules. Reduce to one instance.

In [9]:
shortest_path_overlap_cleaned = {key1: {key2: [] for key2 in GI_shortest_path_dict[key1].keys()} for key1 in GI_shortest_path_dict.keys()}

In [10]:
for key1 in shortest_path_overlap.keys():
    print(key1)
    
    # Skip if there are no entries for this GI clusters (case of 112th cluster).
    if len(shortest_path_overlap[key1]) == 0:
        del shortest_path_overlap_cleaned[key1]
        continue
    
    # If this key contains no shortest paths at all, remove all its nested entries.
    # Note: length > 0 is necessary to avoid error from 112th cluster which is completely empty.
    if (len(reduce(lambda x, y: x+y, shortest_path_overlap[key1].values())) == 0):
        del shortest_path_overlap_cleaned[key1]
        continue
    
    for key2 in shortest_path_overlap[key1].keys():
        if len(shortest_path_overlap[key1][key2]) == 0:
            del shortest_path_overlap_cleaned[key1][key2]
        
        #elif len(shortest_path_overlap[key1][key2]) == 1:
        #    shortest_path_overlap_cleaned[key1][key2] = shortest_path_overlap[key1][key2].copy()
        # Note: If entry is == 1. Then array stays the same because no duplicates are possible.
        
        elif len(shortest_path_overlap[key1][key2]) >= 1:
            shortest_path_overlap_cleaned[key1][key2] = np.unique(shortest_path_overlap[key1][key2].copy())

1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68
69
70
71
72
73
74
75
76
77
78
79
80
81
82
83
84
85
86
87
88
89
90
91
92
93
94
95
96
97
98
99
100
101
102
103
105
106
107
108
109
110
111
112


In [11]:
all_connectors = []
for key1 in shortest_path_overlap_cleaned.keys():
    for key2 in shortest_path_overlap_cleaned[key1].keys():
        if len(shortest_path_overlap_cleaned[key1][key2]) > 0:
            all_connectors.extend(shortest_path_overlap_cleaned[key1][key2])

len(all_connectors)

555

In [12]:
all_unique_connectors = list(np.unique(all_connectors.copy()))
len(all_unique_connectors)

115

# Convert Clean Overlap to Dataframe

In [13]:
## Make a list of lists containing shortest path overlap data
shortest_path_data_ls = []
for key1 in shortest_path_overlap_cleaned.keys():
    for key2 in shortest_path_overlap_cleaned[key1].keys():
        for path in shortest_path_overlap_cleaned[key1][key2]:
            shortest_path_data_ls.append([key1, key2, path.copy()])

In [14]:
## Convert list of lists to dataframe
Shortest_Path_Overlap_DF = pd.DataFrame(shortest_path_data_ls, columns = ['Source_GI_Cluster', 'Target_GI_Cluster', 'Shortest_Path_Overlap'])

## Add Protein Complex IDs
Shortest_Path_Overlap_DF['Source_CPX_ID'] = Shortest_Path_Overlap_DF['Source_GI_Cluster'].apply(lambda x: conversion_dict[x])
Shortest_Path_Overlap_DF['Target_CPX_ID'] = Shortest_Path_Overlap_DF['Target_GI_Cluster'].apply(lambda x: conversion_dict[x])
Shortest_Path_Overlap_DF

,Source_GI_Cluster,Target_GI_Cluster,Shortest_Path_Overlap,Source_CPX_ID,Target_CPX_ID
0,1,38,3541,CPX-1625,CPX-2911
1,2,15,1726,CPX-535,CPX-737
2,2,18,5637,CPX-535,CPX-1862
3,2,20,5228,CPX-535,CPX-1342
4,2,35,1125,CPX-535,CPX-301
...,...,...,...,...,...
550,94,110,4875,CPX-1869,CPX-2948
551,99,100,1235,CPX-1863,CPX-3158
552,99,102,1235,CPX-1863,CPX-1197
553,108,110,2235,CPX-1717,CPX-2948


In [15]:
# Convert ORF integer IDs to ORF systematic IDs (characters)
Shortest_Path_Overlap_DF['Shortest_Path_Overlap'] = Shortest_Path_Overlap_DF['Shortest_Path_Overlap'].apply(lambda x: LUT[x])
Shortest_Path_Overlap_DF

,Source_GI_Cluster,Target_GI_Cluster,Shortest_Path_Overlap,Source_CPX_ID,Target_CPX_ID
0,1,38,YKR018C,CPX-1625,CPX-2911
1,2,15,YER165W,CPX-535,CPX-737
2,2,18,YPL036W,CPX-535,CPX-1862
3,2,20,YOR020C,CPX-535,CPX-1342
4,2,35,YDR182W,CPX-535,CPX-301
...,...,...,...,...,...
550,94,110,YNL243W,CPX-1869,CPX-2948
551,99,100,YDR293C,CPX-1863,CPX-3158
552,99,102,YDR293C,CPX-1863,CPX-1197
553,108,110,YGR097W,CPX-1717,CPX-2948


In [16]:
Shortest_Path_Overlap_DF.to_csv((directory+"Python_Connector_Overlap/results/Shortest_Path_Connector_Overlap.csv"))